# Part 1: Data massaging and enablement

In this jupyter notebook, I used selenium to webscrape the publicly available SpaceX data and structured it into a pandas dataframe which will be used to visualize any trends in the data and answer some analytics questions in a second part. 

## Initiate dependencies and open references

In [1]:
#Import corresponding libraries for selenium webscraping.
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
import pandas as pd
import sqlite3
import copy
import re

In [ ]:
#Initiate chrome driver by calling an instantaneously installed version of the chrome driver manager needed.
driver = webdriver.Chrome(ChromeDriverManager().install())

In [3]:
#Import data from SpaceX wiki website.
driver.get('https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches')

## Data Exploration 

In [4]:
#Let's have a look at our data.
driver.find_element_by_xpath("//table[contains(@class, 'wikitable plainrowheaders collapsible mw-collapsible mw-made-collapsible')][1]/tbody/tr").text

'hide\nFlight No. Date and\ntime (UTC) Version,\nBooster [b] Launch site Payload[c] Payload mass Orbit Customer Launch\noutcome Booster\nlanding'

As we can see, the data seems to be unstructured, but how much? Below I show a snippet of the table format of what I consider to be the 'training set' directly from the wikipedia website, which as we can see has some challenging configurations that will be handled in the process.

![SpaceX_Wiki](spacex_screenshot.png)

Similarly, below I show what I consider to be the 'test set', which introduced further steps in the cleaning process, but was nonetheless accomplished. 

![SpaceX_tWiki](spacex_tscreenshot.png)

Done in a two step process, I first define a check_missing function which is used as a reference function in the main scrape function to verify that the iterations performed have been completely exhausted and there is in fact no more data to be added. 

In [5]:
#Performing as a checking process for the main scrape function,
#this function will ensure whether there is more data to collect, 
#or if in fact it has all been exhausted.
def check_missing(missing_string, number):
    #Parsed as a string,
    #the first step for this function is to fragment it and place it 
    #temporarily in a list of individual elements.
    missing_string_list = list(missing_string)
    
    #Once the string is segmented into the individual elements, 
    #the next thing to do is to customize it accordingly to the iteration number
    #through a set of conditions of either belonging to a single or double integer iteration.
    
    #Case of a single integer iteration.
    if number < 10:
        try:
            #Belonging to a single integer iteration,
            #the modification to the string is made accordingly by pinpointing 
            #the element cointaining that integer and adding one to see ahead.
            missing_string_list[-2] = str(int(missing_string[-2]) + 1)
            
            #Once modified,
            #the list of elements is regrouped to once again make up 
            #the string that will be parsed.
            missing_string = ''.join(missing_string_list)
            
            #Finally, we look ahead and return the result to the main scrape function.
            missing = driver.find_element_by_xpath(missing_string).text
            return(missing)
        
        #It could be the case that nothing passed when calling this function,
        #so to accomodate this I have placed an exception that returns an empty string.
        except NoSuchElementException:
            print('NoSuchElementException from single function')
            return('')
    #Case of a double integer iteration.
    else:
        try:
            #Belonging to a double integer iteration,
            #the modification to the string is made accordingly by pinpointing 
            #the elements cointaining that integer and adding one to see ahead.
            st = str(int(''.join(missing_string_list[-3:-1]))+1)
            
            #Given that there is one too many elements in this list of string elements,
            #this will eliminate it in order to return a properly adjusted string.
            missing_string_list.pop(-2)
            
            #Quick substitution back into the proper element.
            missing_string_list[-2] = st
            
            #Once modified, 
            #the list of elements is regrouped to once again make up
            #the string that will be parsed.
            missing_string = ''.join(missing_string_list)
            
            #Finally, we look ahead and return the result to the main scrape function.
            missing = driver.find_element_by_xpath(missing_string).text
            return(missing)
        
        #It could be the case that nothing passed when calling this function,
        #so to accomodate this I have placed an exception that returns an empty string.
        except NoSuchElementException:
            print('NoSuchElementException from double function')
            return('') 

Next, I define the main scrape main function which contains a set of nested while loops to extract the respective tables, rows and elements from the available data in the wiki site. Cleaning the data through regular expression filsters and customized element replacements, it is then stored in lists which will later be further cleaned and passed onto the more formal pandas dataframe.

In [6]:
def scrape(pre_string):
    #Define the main list, which will store the tables.
    tmain = []
    i = 1  # table
    ttest = True
    j = 1   # row
    test = True
    k = 1   # column
    t2 = True
    #Define the first while loop that will iterate and exhaust the tables available.
    while(ttest):
        try:
            #Define the string that will look for the tables.
            ttest_str = pre_string + str(i) + "]"
            #Store the text result of the ith table.
            ttest = driver.find_element_by_xpath(ttest_str).text
            #Define the sub main list, which will store the rows.
            t_smain = []
            #Define the second while loop that will iterate and exhuast the rows available.
            while(test):
                try:
                    #Define the string that will look for the rows.
                    test_str = pre_string + str(i) + "]/tbody/tr[" + str(j) + "]"
                    #Store the text result of the jth row.
                    test = driver.find_element_by_xpath(test_str).text
                    #Define the sub sub main list, which will store the elements.
                    t_submain = []
                    #It could be possible that there is no data available in the rows, 
                    #in order to accomodate I adjust through a corresponding condition.
                    if test == '':
                        print('Checking main missing.')
                        #Parsing to the previously defined check_missing function, 
                        #here the rows are assured to have data accordingly.
                        t_check = check_missing(test_str, j)
                        #Once again, it could be that only the current row has missing data,
                        #a condition will properly compensate for either case.
                        #This first check is for the case that row has data.
                        if t_check:
                            print(' ')
                            print('False Alarm: Data Continues!')
                            print(' ')
                            #Data could exist, 
                            #so the iteration needs is pushed onto the next row.
                            j += 1
                            test = t_check
                            #Define the third loop that will iterate and exhaust the elements available.
                            while(t2):
                                try: 
                                    #Define the string that will look for the elements.
                                    t2_str = pre_string + str(i) + "]/tbody/tr[" + str(j) + "]/td[" + str(k) + "]"
                                    #Store the text result of the kth element.
                                    t2 = driver.find_element_by_xpath(t2_str).text
                                    #While we are looping to extract the data, an initial cleanup step can be done.
                                    t2 = re.sub(r'\[.*?\]','', t2)            # Remove the citation indices.
                                    t2 = t2.replace('\n','')                  # Remove explicit new line characters.
                                    t2 = t2.replace('°', ' degrees')          # Replace degree symbol with actual word.
                                    t2 = t2.replace('♺', '')                  # Remove recycling symbol from the data.
                                    print(t2)
                                    #Having drilled down to each cell, 
                                    #the data is appended to the previously defined sub list.
                                    t_submain.append(t2) 
                                    #It could be the case that there is no element,
                                    #so we need to check whether in fact it is truly the end of the row.
                                    if t2 == '':
                                        print('Checking missing.')
                                        #Sending the element data once again to the check_missing function,
                                        #this will bring back the corresponding result.
                                        t2_check = check_missing(t2_str, k)
                                        #Conditionally,
                                        #we can once again filter the result accordingly.
                                        if t2_check:
                                            #Replacing the element with the updated version.
                                            t2 = t2_check
                                            #Similar to the steps taken prior to the check,
                                            #if the data exists, a cleaning step can be made.
                                            t2 = re.sub(r'\[.*?\]','', t2)   # Remove the citation indices.
                                            t2 = t2.replace('\n','')         # Remove explicit new line characters.
                                            t2 = t2.replace('°', ' degrees') # Replace degree symbol with actual word.
                                            t2 = t2.replace('♺', '')         # Remove recycling symbol from the data.
                                            #After making sure that each element has data available,
                                            #we can finally append it to the previously defined sub sub list.
                                            t_submain.append(t2)
                                        else:
                                            #It could be the case that there is no more data,
                                            #so the element variable needs to be prepared for the next iteration.
                                            t2 = True
                                            #Additionally, 
                                            #an exception that ends this iteration's execution is performed.
                                            print('NoSuchElementException Raise')
                                            raise NoSuchElementException
                                            break
                                #Once exhausted, 
                                #the element data iterations are ended through an exception.
                                except NoSuchElementException:
                                    print('Inside NoSuchElementException')
                                    break
                                #Element index counter.
                                k += 1
                            #Row index counter.
                            j += 1
                            #Element index reconfiguration.
                            k=1
                            print('')
                        #Contrary to the row containing data case after verification, 
                        #the opposite is performed.
                        else:
                            t_submain.append('')
                            print('Main NoSuchElementException Raise')
                            raise NoSuchElementException
                            break
                    #From the start though, 
                    #it could also be the case that row data is available,
                    #for which this condition would be chosen.
                    else: 
                        #Defining the third loop that will iterate and exhaust the elements available.
                        while(t2):
                            try: 
                                #Defining the string that will look for the elements.
                                t2_str = pre_string + str(i) + "]/tbody/tr[" + str(j) + "]/td[" + str(k) + "]"
                                #Store the text result of the kth element.
                                t2 = driver.find_element_by_xpath(t2_str).text
                                #While we are looping to extract the data, an initial cleanup step can be done.
                                t2 = re.sub(r'\[.*?\]','', t2)            # Remove the citation indices.
                                t2 = t2.replace('\n','')                  # Remove explicit new line characters.
                                t2 = t2.replace('°', ' degrees')          # Replace degree symbol with actual word.
                                t2 = t2.replace('♺', '')                  # Remove recycling symbol from the data.
                                print(t2)
                                #Having drilled down to each cell, 
                                #the data is appended to the previously defined sub list.
                                t_submain.append(t2)
                                #Again, it could be the case that there is no element,
                                #so we need to check whether in fact it is truly the end of the row.
                                if t2 == '':
                                    print('Checking missing.')
                                    #Sending the element data once again to the check_missing function,
                                    #this will bring back the corresponding result.
                                    t2_check = check_missing(t2_str, k)
                                    #Conditionally,
                                    #we can once again filter the result accordingly.
                                    if t2_check:
                                        #Replacing the element with the updated version.
                                        t2 = t2_check
                                        #Similar to the steps taken prior to the check,
                                        #if the data exists, a cleaning step can be made.
                                        t2 = re.sub(r'\[.*?\]','', t2)      # Remove the citation indices.
                                        t2 = t2.replace('\n','')            # Remove explicit new line characters.
                                        t2 = t2.replace('°', ' degrees')    # Replace degree symbol with actual word.
                                        t2 = t2.replace('♺', '')            # Remove recycling symbol from the data.
                                        #After making sure that each element has data available,
                                        #we can finally append it to the previously defined sub sub list.
                                        t_submain.append(t2)
                                    else: 
                                        #It could be the case that there is no more data,
                                        #so the element variable needs to be prepared for the next iteration.
                                        t2 = True
                                        #Additionally, 
                                        #an exception that ends this iteration's execution is performed.
                                        print('NoSuchElementException Raise')
                                        raise NoSuchElementException
                                        break
                            #Once exhausted, 
                            #the element data iterations are ended through an exception.
                            except NoSuchElementException:
                                break
                            #Element index counter.
                            k += 1
                        #Row index counter.
                        j += 1
                        #Element index reconfiguration.
                        k=1
                        print('')
                #Once exhausted,
                #the row data iterations are ended through an exception.
                except NoSuchElementException:
                    break
                #Appending to the row list.
                t_smain.append(t_submain)
            #Table index counter.
            i += 1
            #Row index reconfiguration.
            j = 1
            print(' ')
        #Once exhausted,
        #the table data iterations are ended through an exception.
        except NoSuchElementException:
            break
        #Finally, the cumulative data of list of elements and list of rows are appended to the main list.
        tmain.append(t_smain)
    return(tmain)

In [7]:
#Gather the training data.
train_main = scrape("//table[contains(@class, 'wikitable plainrowheaders collapsible mw-collapsible mw-made-collapsible')][")


4 June 2010,18:45
F9 v1.0B0003.1
CCAFS,SLC-40
Dragon Spacecraft Qualification Unit

Checking missing.
LEO
SpaceX
Success
Failure(parachute)

First flight of Falcon 9 v1.0. Used a boilerplate version of Dragon capsule which was not designed to separate from the second stage.(more details below) Attempted to recover the first stage by parachuting it into the ocean, but it burned up on reentry, before the parachutes even deployed.

8 December 2010,15:43
F9 v1.0B0004.1
CCAFS,SLC-40
Dragon demo flight C1(Dragon C101)

Checking missing.
LEO (ISS)
NASA (COTS)NRO
Success
Failure(parachute)

Maiden flight of Dragon capsule, consisting of over 3 hours of testing thruster maneuvering and reentry. Attempted to recover the first stage by parachuting it into the ocean, but it disintegrated upon reentry, before the parachutes were deployed. (more details below) It also included two CubeSats, and a wheel of Brouère cheese.

22 May 2012,07:44
F9 v1.0B0005.1
CCAFS,SLC-40
Dragon demo flight C2+(Dragon C

F9 v1.1B1015
Cape Canaveral,LC-40
SpaceX CRS-6(Dragon C108.1)
1,898 kg (4,184 lb)
LEO (ISS)
NASA (CRS)
Success
Failure(drone ship)

After second-stage separation, a controlled-descent test was attempted with the first stage. After the booster contacted the ship, it tipped over due to excess lateral velocity caused by a stuck throttle valve that delayed downthrottle at the correct time.

27 April 2015,23:03
F9 v1.1B1016
Cape Canaveral,LC-40
TürkmenÄlem 52 degreesE / MonacoSAT
4,707 kg (10,377 lb)
GTO
Turkmenistan NationalSpace Agency
Success
No attempt

Original intended launch was delayed over a month after an issue with the helium pressurisation system was identified on similar parts in the assembly plant. Subsequent launch successfully positioned this first Turkmen satellite at 52.0 degrees east.

28 June 2015,14:21
F9 v1.1B1018
Cape Canaveral,LC-40
SpaceX CRS-7(Dragon C109)
1,952 kg (4,303 lb)
LEO (ISS)
NASA (CRS)
Failure(in flight)
Precluded(drone ship)

Launch performance was nomi

F9 FTB1031.1
KSC,LC-39A
SpaceX CRS-10(Dragon C112.1)
2,490 kg (5,490 lb)
LEO (ISS)
NASA (CRS)
Success
Success(ground pad)

First Falcon 9 flight from the historic LC-39A launchpad at Kennedy Space Center, and first uncrewed launch from LC-39A since Skylab-1. The flight carried supplies and materials to support ISS Expeditions 50 and 51, and third return of first stage booster to landing pad at Cape Canaveral Landing Zone 1.

16 March 2017,06:00
F9 FTB1030.1
KSC,LC-39A
EchoStar 23
5,600 kg (12,300 lb)
GTO
EchoStar
Success
No attempt

First uncrewed non-station launch from LC-39A since Apollo 6. Launched a communications satellite for broadcast services over Brazil. Due to the payload size launch into a GTO, the booster was expended into the Atlantic Ocean and did not feature landing legs and grid fins.

30 March 2017,22:27
F9 FT B1021.2
KSC,LC-39A
SES-10
5,300 kg (11,700 lb)
GTO
SES
Success
Success(drone ship)

First payload to fly on a reused first stage, B1021, previously launched wit

Northrop Grumman 
Success
Success(ground pad)

The mission had been postponed by nearly two months. Following a nominal launch, the recovery of the first-stage booster marked the 17th successful recovery in a row. Rumors appeared that the payload was lost, as the satellite might have failed to separate from the second stage due to a fault in the Northrop Grumman-manufactured payload adapter, to which SpaceX announced that their rocket performed nominally. The classified nature of the mission means that there is little confirmed information.(more details below)

31 January 2018,21:25
F9 FT B1032.2
CCAFS,SLC-40
GovSat-1 (SES-16)
4,230 kg (9,330 lb)
GTO
SES
Success
Controlled(ocean)

Reused booster from the classified NROL-76 mission in May 2017. Following a successful experimental soft ocean landing that used three engines, the booster unexpectedly remained intact. Recovery was talked about and a Craigslist ad believed to be made by Elon Musk jokingly said the booster was for sale at US$

F9 B5B1047.1
CCAFS,SLC-40
Telstar 19V
7,075 kg (15,598 lb)
GTO
Telesat
Success
Success(drone ship)

SSL-manufactured communications satellite intended to be placed at 63.0 degrees west over the Americas, replacing Telstar 14R. At 7,075 kg (15,598 lb), it became the heaviest commercial communications satellite so far launched. This necessitated that the satellite be launched into a lower-energy orbit than a usual GTO, with its initial apogee at roughly 17,900 km (11,100 mi).

25 July 2018,11:39
F9 B5B1048.1
VAFB,SLC-4E
Iridium NEXT-7(10 satellites)
9,600 kg (21,200 lb)
Polar LEO
Iridium Communications
Success
Success(drone ship)

Seventh Iridium NEXT launch, with 10 communication satellites. The booster landed safely on the drone ship in the worst weather conditions for any landing yet attempted. Mr. Steven boat with an upgraded 4x size net was used to attempt fairing recovery but failed due to harsh weather.

7 August 2018,05:18
F9 B5 B1046.2
CCAFS,SLC-40
Merah Putih (formerly Telkom 4

VAFB,SLC-4E
RADARSAT Constellation(3 satellites)
4,200 kg (9,300 lb)
SSO
Canadian Space Agency (CSA)
Success
Success(ground pad)

A trio of satellites built for Canada's RADARSAT program were launched that plan to replace the aging Radarsat-1 and Radarsat-2. The new satellites contain Automated Identification System (AIS) for locating ships and provide the world's most advanced, comprehensive method of maintaining Arctic sovereignty, conducting coastal surveillance, and ensuring maritime security. The mission was originally scheduled to lift off in February but due to the landing failure of booster B1050, this flight was switched to B1051 (used on Crew Dragon Demo-1) and delayed to allow refurbishment and transport to the West coast. The booster landed safely through fog. A payload cost of roughly US$1 billion made this SpaceX's second most expensive payload launched and most valuable commercial payload so far put into orbit.

25 June 2019,06:30
Falcon HeavyB1057 core
KSC,LC-39A
Space 

LEO
SpaceX
Success
Failure(drone ship)

Fourth operational and fifth large batch of Starlink satellites. Used a new flight profile which deployed into a 212 km × 386 km (132 mi × 240 mi) elliptical orbit instead of launching into a circular orbit and firing the second stage engine twice. The first stage booster failed to land on the drone ship due to incorrect wind data. This was the first time a flight proven booster failed to land.

7 March 2020,04:50
F9 B5 B1059.2
CCAFS,SLC-40
SpaceX CRS-20(Dragon C112.3 )
1,977 kg (4,359 lb)
LEO (ISS)
NASA (CRS)
Success
Success(ground pad)

Last launch of phase 1 of the CRS contract. Carries Bartolomeo, an ESA platform for hosting external payloads onto ISS. Originally scheduled to launch on 2 March 2020, the launch date was pushed back due to a second stage engine failure. SpaceX decided to swap out the second stage instead of replacing the faulty part. It was SpaceX's 50th successful landing of a first stage booster, the third flight of the Drago

24 October 202015:31:34
F9 B5 B1060.3
CCAFS,SLC-40
Starlink 14 v1.0 (60 satellites)
15,600 kg (34,400 lb)
LEO
SpaceX
Success
Success(drone ship)

Fourteenth operational launch of Starlink satellites and the 100th successful launch of a Falcon vehicle.

5 November 202023:24:23
F9 B5B1062.1
CCAFS,SLC-40
GPS III-04 (Sacagawea)
4,311 kg (9,504 lb)
MEO
USSF
Success
Success(drone ship)

Manufacturing contract awarded in January 2012, underwent thermal vacuum testing in December 2018, while the launch contract was awarded in March 2018. A launch attempt on 3 October 2020 was aborted two seconds before liftoff due to early start in two engines. Following the abort, two engines from B1062 were sent for further testing. The abort also caused delays to the Crew-1 launch to allow time for data review.

16 November 202000:27
F9 B5B1061.1
KSC,LC-39A
Crew-1(Crew Dragon C207.1 Resilience)
~12,500 kg (27,600 lb)
LEO (ISS)
NASA (CCP)
Success
Success(drone ship)

First crew rotation of the commercial cre

24 March 202108:28
F9 B5 B1060.6
CCSFS,SLC-40
Starlink 22 v1.0 (60 satellites)
15,600 kg (34,400 lb)
LEO
SpaceX
Success
Success(drone ship)

Fairing "wet recovery" achieved by contracted recovery vessel Shelia Bordelon for the first time. Both fairing halves were retrieved from the water.

7 April 202116:34
F9 B5 B1058.7
CCSFS,SLC-40
Starlink 23 v1.0 (60 satellites)
15,600 kg (34,400 lb)
LEO
SpaceX
Success
Success(drone ship)

23rd operational launch of Starlink satellites, bringing the total to 1,585 launched Starlink satellites (including prototype). This launch featured the second fastest booster turnaround time at 27 days and 8 hours (after Starlink 18 with B1049.8, which was 4 hours faster).

 


In [8]:
#Gather the testing data.
test_main = scrape("//table[@class='wikitable'][")


22 April 202110:11
F9 B5 B1061.2
KSC,LC-39A
Crew-2(Crew Dragon C206-2 Endeavour )
LEO (ISS)
NASA (CTS)

Second operational flight of Crew Dragon for Commercial Crew Program. Will transport NASA astronauts Shane Kimbrough and Megan McArthur, JAXA Astronaut Akihiko Hoshide and ESA astronaut Thomas Pesquet to the ISS. The four astronauts will spend 6 months aboard the ISS. Starting on Crew-2 SpaceX is allowed to fly astronauts in re-used Dragon capsules with reused boosters. Thus SpaceX plans to refly the Dragon used on Demo-2 and use Booster B1061-2 which has been used to launch Crew-1 in November 2020.

April 2021
F9 B5
CC,LC-39A or SLC-40
Starlink 24
LEO
SpaceX

Checking main missing.
 
False Alarm: Data Continues!
 
April 2021
F9 B5
CC,LC-39A or SLC-40
Starlink possibly with smallsat rideshare
LEO
SpaceX
Inside NoSuchElementException

Checking main missing.
 
False Alarm: Data Continues!
 
May 2021
F9 B5
CC,LC-39A or SLC-40
Starlink possibly with smallsat rideshare
LEO
SpaceX
Inside 

SpaceX

Checking main missing.
 
False Alarm: Data Continues!
 
December 2021
F9 B5
CCSFS,SLC-40
NROL-85 (Intruder 13A and 13B)
LEO
NRO
Inside NoSuchElementException

Classified mission awarded to SpaceX in February 2019.

December 2021
F9 B5
VAFB,SLC-4E
NROL-87
SSO
NRO

Classified payload

4 December 2021
F9 B5
KSC,LC-39A
SpaceX CRS-24
LEO (ISS)
NASA (CRS)

In 2015, NASA awarded SpaceX a minimum of six new cargo missions under the CRS2 contract after the initial 20 missions of phase 1, which will be flown with an uncrewed Dragon 2 capsule.

December 2021
F9 B5
VAFB,SLC-4E
Transporter-3, SmallSat Rideshare
SSO
Various

Momentus has reserved payloads on five SSO launches.

December 2021
F9 B5 
VAFB,SLC-4E
WorldView Legion Mission 2
SSO

Checking missing.
NoSuchElementException from single function
NoSuchElementException Raise

Maxar Technologies built satellites.

Q4 2021 to mid 2022
F9 B5
KSC,LC-39A
Space Adventures Dragon Mission
LEO
Space Adventures

SpaceX signed in February 2020, i

Griffin Mission 1
TLI
Astrobotic/NASA (Artemis)

Astrobotic's Griffin lunar lander will deliver NASA's VIPER spacecraft to the lunar south pole.

2023
F9 B5
CC,LC-39A or SLC-40
Intelsat satellite
GTO
Intelsat

Intelsat contracted both SpaceX and Arianespace to launch its fifth Maxar Technologies built satellite, and award whichever doesn't launch it with a separate contract at a later date.

2023–2026(2 launches)
F9 B5 
KSC,LC-39A
Crew-5 and Crew-6
LEO (ISS)
NASA (CTS)

Two more USCV launches out of NASA award of six Crew Dragon mission, to carry up to four astronauts and 100 kg (220 lb) of cargo to the ISS as well as feature a lifeboat function to evacuate astronauts from ISS in case of an emergency.

2023 Q4
F9 B5
CC,LC-39A or SLC-40
SATRIA
GTO
PT Pasifik Satelit Nusantara

PSN chose Falcon 9 in September 2020 to launch its satellite instead of a Chinese rocket or Ariane 5.

2023 Q4
F9 B5
CC,LC-39A or SLC-40
USSF-36
TBD
USSF

Launch part of Phase 2 US Air Force contract awarded in 20

## Data cleanup

Now that the data is in a structured format that I can work with and has gone through an inital cleaning process, the next step is to further organize the data into something that can eventually end up as a dataframe that can be used for the analysis part. Before working on that process, I first had to retrieve the most important part needed to get to the end goal - the column titles. 

In [9]:
#First, since the title row was ignored in the previous nested loop (due to it having a different path from the data),
#I will do this first.
t = 1
title = True
title_list = []
while(title):
    try:
        title_str = "//table[contains(@class, 'wikitable plainrowheaders collapsible mw-collapsible mw-made-collapsible')][1]/tbody/tr/th[" + str(t) + "]"
        title = driver.find_element_by_xpath(title_str).text
        title = title.replace('\n', ' ')
        title = title.replace('hide', '')
        title = re.sub(r'\[.*?\]','', title) 
        title_list.append(title)
        t += 1
    except NoSuchElementException:
        break
title_list = title_list[1:] + ['Description']
title_list

['Date and time (UTC)',
 'Version, Booster ',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome',
 'Booster landing',
 'Description']

In [366]:
#After that hard work, let's copy to be more efficient.
spacex_main = copy.deepcopy(train_main)
spacex_test = copy.deepcopy(test_main)

In [367]:
spacex_main

[[[],
  ['4 June 2010,18:45',
   'F9 v1.0B0003.1',
   'CCAFS,SLC-40',
   'Dragon Spacecraft Qualification Unit',
   '',
   'LEO',
   'LEO',
   'SpaceX',
   'Success',
   'Failure(parachute)'],
  ['First flight of Falcon 9 v1.0. Used a boilerplate version of Dragon capsule which was not designed to separate from the second stage.(more details below) Attempted to recover the first stage by parachuting it into the ocean, but it burned up on reentry, before the parachutes even deployed.'],
  ['8 December 2010,15:43',
   'F9 v1.0B0004.1',
   'CCAFS,SLC-40',
   'Dragon demo flight C1(Dragon C101)',
   '',
   'LEO (ISS)',
   'LEO (ISS)',
   'NASA (COTS)NRO',
   'Success',
   'Failure(parachute)'],
  ['Maiden flight of Dragon capsule, consisting of over 3 hours of testing thruster maneuvering and reentry. Attempted to recover the first stage by parachuting it into the ocean, but it disintegrated upon reentry, before the parachutes were deployed. (more details below) It also included two CubeSa

In [368]:
spacex_test

[[[],
  ['22 April 202110:11',
   'F9 B5 B1061.2',
   'KSC,LC-39A',
   'Crew-2(Crew Dragon C206-2 Endeavour )',
   'LEO (ISS)',
   'NASA (CTS)'],
  ['Second operational flight of Crew Dragon for Commercial Crew Program. Will transport NASA astronauts Shane Kimbrough and Megan McArthur, JAXA Astronaut Akihiko Hoshide and ESA astronaut Thomas Pesquet to the ISS. The four astronauts will spend 6 months aboard the ISS. Starting on Crew-2 SpaceX is allowed to fly astronauts in re-used Dragon capsules with reused boosters. Thus SpaceX plans to refly the Dragon used on Demo-2 and use Booster B1061-2 which has been used to launch Crew-1 in November 2020.'],
  ['April 2021',
   'F9 B5',
   'CC,LC-39A or SLC-40',
   'Starlink 24',
   'LEO',
   'SpaceX'],
  ['April 2021',
   'F9 B5',
   'CC,LC-39A or SLC-40',
   'Starlink possibly with smallsat rideshare',
   'LEO',
   'SpaceX'],
  ['May 2021',
   'F9 B5',
   'CC,LC-39A or SLC-40',
   'Starlink possibly with smallsat rideshare',
   'LEO',
   'Spa

Having the titles available at my disposal, I could now start with the assembly of the data and form a precursor of the end dataframe using lists. Accomodating the rows first that require complementary data in order to build the dataframe, this is the initial step taken below:

In [369]:
#Nested list comprehension to see what rows in each table of the training set are incomplete.
[[i for i in spacex_main[j] if len(i) < 9 and len(i) > 1] for j in range(len(spacex_main))]

[[['Orbcomm-OG2', '172 kg (379 lb)', 'LEO', 'Orbcomm', 'Partial failure']],
 [],
 [],
 [],
 [],
 [['B1023.2 (side) ', 'Success(ground pad)'],
  ['B1025.2 (side) ', 'Success(ground pad)']],
 [['B1052.1(side)', 'Success(ground pad)'],
  ['B1053.1(side)', 'Success(ground pad)'],
  ['B1052.2(side) ', 'Success(ground pad)'],
  ['B1053.2(side) ', 'Success(ground pad)']],
 [],
 []]

In [370]:
#Nested list comprehension to see what rows in each table of the test set are incomplete.
[[i for i in spacex_test[j] if len(i) < 4 and len(i) > 1] for j in range(len(spacex_test))]

[[], [], []]

As we can see, there are 7 rows that still require data in order for the row to be complete (i.e. be the same size as the rest of the rows), so we can modify them manually using a list concatenation method. It is possible to automate this process, but it is challenging since it is not straight forward to figure out which index is the one that is correctly assigned to a specific element (e.g. B1023.2 (side) may belong to element 1 or 2, etc.). Alternatively, this could also be semi-automated using user input to handle the modification, but since this isn't much data a full manual modification is performed.

In [371]:
#First row modification containing flight attributes.
spacex_main[0][8] = spacex_main[0][7][0:3] + spacex_main[0][8][0:5] + [spacex_main[0][7][-1]]
#Duplicate and place description for flight in appropriate location.
spacex_main[0].insert(8,spacex_main[0][9])

In [372]:
#Second row modification containing flight attributes.
spacex_main[5][6] = [spacex_main[5][5][0]] + [spacex_main[5][6][0]] + spacex_main[5][5][2:8] + [spacex_main[5][6][1]]
#Duplicate and place description for flight in appropriate location.
spacex_main[5].insert(6,spacex_main[5][8])

In [373]:
#Third row modification containing flight attributes.
spacex_main[5][8] = [spacex_main[5][5][0]] + [spacex_main[5][8][0]] + spacex_main[5][5][2:8] + [spacex_main[5][8][1]]
#Duplicate and place description for flight in appropriate location.
spacex_main[5].insert(8,spacex_main[5][9])

In [374]:
#Fourth row modification containing flight attributes.
spacex_main[6][8] = [spacex_main[6][7][0]] + [spacex_main[6][8][0]] + spacex_main[6][7][2:8] + [spacex_main[6][8][1]]
#Duplicate and place description for flight in appropriate location.
spacex_main[6].insert(8,spacex_main[6][10])

In [375]:
#Fifth row modification containing flight attributes.
spacex_main[6][10] = [spacex_main[6][7][0]] + [spacex_main[6][10][0]] + spacex_main[6][7][2:8] + [spacex_main[6][10][1]]
#Duplicate and place description for flight in appropriate location.
spacex_main[6].insert(10,spacex_main[6][11])

In [376]:
#Sixth row modification containing flight attributes.
spacex_main[6][20] = [spacex_main[6][19][0]] + [spacex_main[6][20][0]] + spacex_main[6][19][2:8] + [spacex_main[6][20][1]]
#Duplicate and place description for flight in appropriate location.
spacex_main[6].insert(20,spacex_main[6][22])

In [377]:
#Seventh row modification containing flight attributes.
spacex_main[6][22] = [spacex_main[6][19][0]] + [spacex_main[6][22][0]] + spacex_main[6][19][2:8] + [spacex_main[6][22][1]]
#Duplicate and place description for flight in appropriate location, ready to iterate!
spacex_main[6].insert(22,spacex_main[6][23])

In [378]:
#Check if any more are missing.
[[i for i in spacex_main[j] if len(i) < 9 and len(i) > 1] for j in range(len(spacex_main))]

[[], [], [], [], [], [], [], [], []]

In [379]:
#Example of the end goal to properly align list for its transformation onto a dataframe.
spacex_main[0][1:3]

[['4 June 2010,18:45',
  'F9 v1.0B0003.1',
  'CCAFS,SLC-40',
  'Dragon Spacecraft Qualification Unit',
  '',
  'LEO',
  'LEO',
  'SpaceX',
  'Success',
  'Failure(parachute)'],
 ['First flight of Falcon 9 v1.0. Used a boilerplate version of Dragon capsule which was not designed to separate from the second stage.(more details below) Attempted to recover the first stage by parachuting it into the ocean, but it burned up on reentry, before the parachutes even deployed.']]

Now that all of the data is properly aligned in the same format as the example above, the next thing to do is to join the data in order to complete the rows and prepare it for a dataframe transformation. So in the case of the list of 2 lists that can be seen above, the result would be a list of 1 list containing all of the data for the datapoint.

In [380]:
#Join data with comments to complete rows.
spacex_main = [[[inner for outer in spacex_main[j][i:i+2] for inner in outer] for i in range(1,int((len(spacex_main[j])-1))) if i % 2 != 0] for j in range(len(spacex_main))]

In [381]:
spacex_test = [[[inner for outer in spacex_test[j][i:i+2] for inner in outer] for i in range(1, int((len(spacex_test[j])-1))) if i % 2 !=0] for j in range(len(spacex_test))]

In [382]:
#Flatten the 3D list to a 2D list, ready for dataframe transform!
spacex_main = [sitem for ssublist in spacex_main for sitem in ssublist]
spacex_test = [sitem for ssublist in spacex_test for sitem in ssublist]
spacex_main[0]

['4 June 2010,18:45',
 'F9 v1.0B0003.1',
 'CCAFS,SLC-40',
 'Dragon Spacecraft Qualification Unit',
 '',
 'LEO',
 'LEO',
 'SpaceX',
 'Success',
 'Failure(parachute)',
 'First flight of Falcon 9 v1.0. Used a boilerplate version of Dragon capsule which was not designed to separate from the second stage.(more details below) Attempted to recover the first stage by parachuting it into the ocean, but it burned up on reentry, before the parachutes even deployed.']

## Data preparation

As part of the last couple of steps, the data is revised once more to see if there are dimensional congruencies. Given that there was some duplicate data, these are removed and the transformation is pushed onto a dataframe. 

In [383]:
#First two datapoints do not have the same dimensions as the rest of the data.
for i in range(len(spacex_main)):
    print(i,len(spacex_main[i]))

0 11
1 11
2 10
3 10
4 10
5 10
6 10
7 10
8 10
9 10
10 10
11 10
12 10
13 10
14 10
15 10
16 10
17 10
18 10
19 10
20 10
21 10
22 10
23 10
24 10
25 10
26 10
27 10
28 10
29 10
30 10
31 10
32 10
33 10
34 10
35 10
36 10
37 10
38 10
39 10
40 10
41 10
42 10
43 10
44 10
45 10
46 10
47 10
48 10
49 10
50 10
51 10
52 10
53 10
54 10
55 10
56 10
57 10
58 10
59 10
60 10
61 10
62 10
63 10
64 10
65 10
66 10
67 10
68 10
69 10
70 10
71 10
72 10
73 10
74 10
75 10
76 10
77 10
78 10
79 10
80 10
81 10
82 10
83 10
84 10
85 10
86 10
87 10
88 10
89 10
90 10
91 10
92 10
93 10
94 10
95 10
96 10
97 10
98 10
99 10
100 10
101 10
102 10
103 10
104 10
105 10
106 10
107 10
108 10
109 10
110 10
111 10
112 10
113 10
114 10
115 10
116 10
117 10
118 10
119 10
120 10
121 10
122 10
123 10


In [384]:
#Upon further inspection,
#it can be seen that the orbit for both is duplicated.
spacex_main[1]

['8 December 2010,15:43',
 'F9 v1.0B0004.1',
 'CCAFS,SLC-40',
 'Dragon demo flight C1(Dragon C101)',
 '',
 'LEO (ISS)',
 'LEO (ISS)',
 'NASA (COTS)NRO',
 'Success',
 'Failure(parachute)',
 'Maiden flight of Dragon capsule, consisting of over 3 hours of testing thruster maneuvering and reentry. Attempted to recover the first stage by parachuting it into the ocean, but it disintegrated upon reentry, before the parachutes were deployed. (more details below) It also included two CubeSats, and a wheel of Brouère cheese.']

In [385]:
#Removal of first duplicate data.
spacex_main[0].pop(5)

'LEO'

In [386]:
#Removal of second duplicate data.
spacex_main[1].pop(5)

'LEO (ISS)'

In [387]:
spacex_main_df = pd.DataFrame(spacex_main, columns = title_list)

In [509]:
spacex_main_df.tail()

,Date and time (UTC),"Version, Booster",Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing,Description
119,4 March 202108:24,F9 B5 B1049.8,"KSC,LC-39A",Starlink 17 v1.0 (60 satellites),"15,600 kg (34,400 lb)",LEO,SpaceX,Success,Success(drone ship),Launch had previously been postponed multiple ...
120,11 March 202108:13:29,F9 B5 B1058.6,"CCSFS,SLC-40",Starlink 20 v1.0 (60 satellites),"15,600 kg (34,400 lb)",LEO,SpaceX,Success,Success(drone ship),Twentieth operational launch of Starlink satel...
121,14 March 202110:01,F9 B5 B1051.9,"KSC,LC-39A",Starlink 21 v1.0 (60 satellites),"15,600 kg (34,400 lb)",LEO,SpaceX,Success,Success(drone ship),First time a first-stage booster flew and land...
122,24 March 202108:28,F9 B5 B1060.6,"CCSFS,SLC-40",Starlink 22 v1.0 (60 satellites),"15,600 kg (34,400 lb)",LEO,SpaceX,Success,Success(drone ship),"Fairing ""wet recovery"" achieved by contracted ..."
123,7 April 202116:34,F9 B5 B1058.7,"CCSFS,SLC-40",Starlink 23 v1.0 (60 satellites),"15,600 kg (34,400 lb)",LEO,SpaceX,Success,Success(drone ship),23rd operational launch of Starlink satellites...


In [502]:
for i in range(len(spacex_test)):
    print(i,len(spacex_test[i]))

0 7
1 7
2 7
3 7
4 7
5 7
6 7
7 7
8 7
9 7
10 7
11 7
12 7
13 7
14 7
15 7
16 7
17 7
18 7
19 7
20 7
21 7
22 7
23 7
24 7
25 7
26 7
27 7
28 7
29 7
30 7
31 7
32 7
33 7
34 7
35 7
36 7
37 7
38 7
39 7
40 7
41 7
42 7
43 7
44 7
45 7
46 7
47 7
48 7
49 7
50 7
51 7
52 7
53 7
54 7
55 7
56 7
57 7
58 7
59 7
60 7
61 7
62 7
63 7
64 7
65 7
66 7
67 7
68 7
69 7
70 7


In [394]:
#Remove duplicates from elements.
spacex_test[6].pop(3)
spacex_test[30].pop(5)

'Inmarsat'

In [396]:
#Separate second element and insert the appropriate third element.
spacex_test.insert(2,spacex_test[1][6:])
spacex_test.insert(4,spacex_test[3][6:])
spacex_test.insert(13,spacex_test[12][6:])
spacex_test.insert(20,spacex_test[19][6:])

In [397]:
#Append the right descriptions to the right elements.
spacex_test[4].append(spacex_test[5][0])
spacex_test[5].append(spacex_test[6][0])
spacex_test[6].append(spacex_test[7][0])
spacex_test[13].append(spacex_test[14][0])
spacex_test[20].append(spacex_test[21][0])
spacex_test[21].append(spacex_test[22][0])
spacex_test[22].append(spacex_test[23][0])
spacex_test[23].append(spacex_test[24][0])
spacex_test[24].append(spacex_test[25][0])
spacex_test[25].append(spacex_test[26][0])
spacex_test[26].append(spacex_test[27][0])

In [398]:
#Delete the third element from the second element.
spacex_test[1] = spacex_test[1][:6]
spacex_test[3] = spacex_test[3][:6]
spacex_test[12] = spacex_test[12][:6]
spacex_test[19] = spacex_test[19][:6]

In [399]:
#Delete the first element from each of the rows that classified incorrectly.
spacex_test[5] = spacex_test[5][1:]
spacex_test[6] = spacex_test[6][1:]
spacex_test[7] = spacex_test[7][1:]
spacex_test[14] = spacex_test[14][1:]
spacex_test[21] = spacex_test[21][1:]
spacex_test[22] = spacex_test[22][1:]
spacex_test[23] = spacex_test[23][1:]
spacex_test[24] = spacex_test[24][1:]
spacex_test[25] = spacex_test[25][1:]
spacex_test[26] = spacex_test[26][1:]
spacex_test[27] = spacex_test[27][1:]

In [501]:
#Insert missing value into second element.
spacex_test[1].append('')
spacex_test[2].append('')
spacex_test[3].append('')
spacex_test[7].append('')
spacex_test[12].append('')
spacex_test[14].append('')
spacex_test[19].append('')
spacex_test[27].append('')

In [507]:
spacex_test_df = pd.DataFrame(spacex_test, columns = ['Date and time (UTC)', 'Version Booster', 'Launch site', 'Payload', 'Orbit', 'Customer', 'Description'])

In [508]:
spacex_test_df

,Date and time (UTC),Version Booster,Launch site,Payload,Orbit,Customer,Description
0,22 April 202110:11,F9 B5 B1061.2,"KSC,LC-39A",Crew-2(Crew Dragon C206-2 Endeavour ),LEO (ISS),NASA (CTS),Second operational flight of Crew Dragon for C...
1,April 2021,F9 B5,"CC,LC-39A or SLC-40",Starlink 24,LEO,SpaceX,
2,April 2021,F9 B5,"CC,LC-39A or SLC-40",Starlink possibly with smallsat rideshare,LEO,SpaceX,
3,May 2021,F9 B5,"CC,LC-39A or SLC-40",Starlink possibly with smallsat rideshare,LEO,SpaceX,
4,1 June 202104:25,F9 B5,"CCSFS,SLC-40",SXM-8,GTO,Sirius XM,"A large, high-power broadcasting satellite for..."
...,...,...,...,...,...,...,...
66,June 2024,F9 B5,"VAFB,SLC-4E",SPHEREx,SSO,NASA,"In February 2021, NASA announced a $99m contra..."
67,2024,F9 B5,"CC,LC-39A or SLC-40",O3b mPOWER 10 and 11,MEO,SES,"In August 2020, SES expanded the O3m contract ..."
68,2024,Falcon Heavy,"KSC,LC-39A",Dragon XL,TLI,NASA (Gateway Logistics Services),"In March 2020, NASA announced its first contra..."
69,1 February 2025,F9 B5,"KSC,LC-39A",Interstellar Mapping and Acceleration Probe (I...,Sun–Earth L1,NASA,"In September 2020, NASA selected SpaceX to lau..."


As a final step, the data will be sent off to a database using python's sqlite3 in order to access it in the second part for the analysis.

In [512]:
#Set up a connection to the sqlite3 server.
conn = sqlite3.connect('spacex_data.db')

In [513]:
#Send the data as a table to the database.
spacex_main_df.to_sql(name='spacex_flights',con=conn, index=False)

In [ ]:
spacex_test_df.to_sql(name='spacex_test', con=conn, index=False)

In [518]:
#Ready!
pd.read_sql_query("SELECT * FROM spacex_test", conn)

,Date and time (UTC),Version Booster,Launch site,Payload,Orbit,Customer,Description
0,22 April 202110:11,F9 B5 B1061.2,"KSC,LC-39A",Crew-2(Crew Dragon C206-2 Endeavour ),LEO (ISS),NASA (CTS),Second operational flight of Crew Dragon for C...
1,April 2021,F9 B5,"CC,LC-39A or SLC-40",Starlink 24,LEO,SpaceX,
2,April 2021,F9 B5,"CC,LC-39A or SLC-40",Starlink possibly with smallsat rideshare,LEO,SpaceX,
3,May 2021,F9 B5,"CC,LC-39A or SLC-40",Starlink possibly with smallsat rideshare,LEO,SpaceX,
4,1 June 202104:25,F9 B5,"CCSFS,SLC-40",SXM-8,GTO,Sirius XM,"A large, high-power broadcasting satellite for..."
...,...,...,...,...,...,...,...
66,June 2024,F9 B5,"VAFB,SLC-4E",SPHEREx,SSO,NASA,"In February 2021, NASA announced a $99m contra..."
67,2024,F9 B5,"CC,LC-39A or SLC-40",O3b mPOWER 10 and 11,MEO,SES,"In August 2020, SES expanded the O3m contract ..."
68,2024,Falcon Heavy,"KSC,LC-39A",Dragon XL,TLI,NASA (Gateway Logistics Services),"In March 2020, NASA announced its first contra..."
69,1 February 2025,F9 B5,"KSC,LC-39A",Interstellar Mapping and Acceleration Probe (I...,Sun–Earth L1,NASA,"In September 2020, NASA selected SpaceX to lau..."
